In [99]:
import glob
import os
import json
import csv
import random
import numpy as np
import pandas as pd


In [100]:
MATCHED_PATH = "lpd_5/lpd_5_matched/"
CLEANED_PATH = "lpd_5/lpd_5_cleansed/"
MUMU_AMAZON_PATH = "MuMu_dataset/amazon_reviews_MuMu.json"
MUMU_MULTI_PATH = "MuMu_dataset/MuMu_dataset_multi-label.csv"
MUMU_SINGLE_PATH = "MuMu_dataset/MuMu_dataset_single-label.csv"

matched_file_paths = []
for x in os.walk(MATCHED_PATH):
    for y in glob.glob(os.path.join(x[0], '*.npz')):
        matched_file_paths.append(y)

cleaned_file_paths = []
for x in os.walk(CLEANED_PATH):
    for y in glob.glob(os.path.join(x[0], '*.npz')):
        cleaned_file_paths.append(y)

In [34]:
# total number of songs
print(len(matched_file_paths), len(cleaned_file_paths))

115160 21425


In [101]:
mumu_multi_df = pd.read_csv(MUMU_MULTI_PATH)
mumu_single_df = pd.read_csv(MUMU_SINGLE_PATH)
mumu_amazon_df = pd.read_json(MUMU_AMAZON_PATH, lines=True)

In [102]:
#list of MSD and recording ids of the piano roll dataset
cleaned_MSD_id_list = []
uncleaned_MSD_id_list = []
recording_mbid_list = []
for i in range(len(cleaned_file_paths)):
    curr_path = file_paths[i]
    curr_MSD_id = curr_path.split('/')[-2]
    cleaned_MSD_id_list.append(curr_MSD_id)
for i in range(len(matched_file_paths)):
    curr_path = file_paths[i]
    curr_MSD_id = curr_path.split('/')[-2]
    uncleaned_MSD_id_list.append(curr_MSD_id)
    recording_mbid_list.append(curr_path.split('/')[-1].split('.')[0])
unique_uncleaned_MSD_id_list = list(set(uncleaned_MSD_id_list))

In [67]:
label_list = []
tuple_list = []
DATA_LENGTH = 20000
NUM_FILE = 100
CLEANED = False

idx = 0
while len(tuple_list) < NUM_FILE:
    MSD_ids = cleaned_MSD_id_list if CLEANED else unique_uncleaned_MSD_id_list
    curr_path = cleaned_file_paths[idx] if CLEANED else matched_file_paths[idx]
    idx += 1
    curr_MSD_id = curr_path.split('/')[-2]
    curr_amazon_id = mumu_multi_df.loc[mumu_multi_df['MSD_track_id'] == curr_MSD_id]
    
    if len(curr_amazon_id) == 0:
        curr_amazon_id = mumu_single_df.loc[mumu_single_df['MSD_track_id'] == curr_MSD_id]
        if len(curr_amazon_id) == 0:
            continue
            
    npz = np.load(curr_path)
    df= pd.DataFrame.from_dict({item: npz[item] for item in npz.files}, orient='index').T
    data_nparr = df[['pianoroll_0_csc_data', 'pianoroll_1_csc_data', 'pianoroll_2_csc_data', 'pianoroll_3_csc_data', 'pianoroll_4_csc_data']].fillna(0).to_numpy()
    if data_nparr.shape[0] > DATA_LENGTH:
        data_nparr = data_nparr[:DATA_LENGTH + 1]
    elif data_nparr.shape[0] < DATA_LENGTH:
        np.pad(data_nparr, (0, DATA_LENGTH - data_nparr.shape[0]), 'constant', constant_values=(0, 0))

    curr_review = mumu_amazon_df.loc[mumu_amazon_df['amazon_id'] == curr_amazon_id['amazon_id'].values[0]]['reviewText']
    curr_tuple = [data_nparr.tolist(), curr_review.values[0]]
    tuple_list.append(curr_tuple)
    label_list.append(curr_review.values[0])


In [88]:
neg_tuple_list = []
for tuples in tuple_list:
    neg_label = tuples[1]
    while neg_label == tuples[1]:
        neg_label = random.choice(label_list)
    neg_tuple = [tuples[0], neg_label]
    neg_tuple_list.append(neg_tuple)

In [89]:
POS_TUPLE_PATH = "pos_tuples_mini.csv"
NEG_TUPLE_PATH = "neg_tuples_mini.csv"
fields = ['data', 'label'] 
with open(POS_TUPLE_PATH, 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(tuple_list)
with open(NEG_TUPLE_PATH, 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(neg_tuple_list)

In [96]:
len(neg_tuple_list)

100

In [111]:
#mintime: 1824
#maxtime: 166926
#avgtime of first 100: 31551.36

#number of cleaned pianoroll tracks: 21425
#cleaned tracks matched to unique labels: 2504 / 1916 / 1701 / 1816
#all matching labels of cleaned tracks: 39776 / 40172

#number of uncleaned pianoroll recordings: 115160
#unique uncleaned tracks matched to unique labels: 3559 / 3328
#all matching labels of unique uncleaned tracks: 76861
#all uncleaned recordings matched to repetative lablels: 11697

#size of entire mumudataset: 956292
# There may be multiple reviews to the same MSD track!!!

#count pianoroll matched to amazon review
count = 0
count_review = 0
for idd in unique_uncleaned_MSD_id_list:
    curr_amazon_id = mumu_single_df.loc[mumu_single_df['MSD_track_id'] == idd]
    if len(curr_amazon_id) == 0:
        curr_amazon_id = mumu_multi_df.loc[mumu_multi_df['MSD_track_id'] == idd]
        if len(curr_amazon_id) == 0:
            continue
    curr_review = mumu_amazon_df.loc[mumu_amazon_df['amazon_id'] == curr_amazon_id['amazon_id'].values[0]]['reviewText']
    count += 1
    count_review += len(curr_review.values)

In [112]:
print(count, count_review)

3328 76861
